In [2]:
import pandas as pd

data_dir = '/home/leiyu/scratch/cma_hallucination/data/'
max_len_prompt = 32

lama_completion_df = pd.read_csv(
        data_dir + 'lama/gpt2-xl-completion-prompt-{}.csv'.format(max_len_prompt))

In [6]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

lama_completion_df.head(50)

,prompt,true object,LM completion
0,The A605 road is a main road in the English counties of,Northamptonshire,"The A605 road is a main road in the English counties of Cumbria,"
1,The A605 road is a main road in the English counties of,Northamptonshire,"The A605 road is a main road in the English counties of Cumbria,"
2,It connects the Leicestershire town of Market Harborough and the A6 with the,Northamptonshire,It connects the Leicestershire town of Market Harborough and the A6 with the ute route between the two
3,It connects the Leicestershire town of Market Harborough and the A6 with the,Northamptonshire,It connects the Leicestershire town of Market Harborough and the A6 with the ute route between the two
4,It connects the Leicestershire town of Market Harborough and the A6 with the,Northamptonshire,It connects the Leicestershire town of Market Harborough and the A6 with the ute route between the two
5,"The Lindenberg Peninsula is a peninsula on the eastern coast of Kupreanof Island, Alexander Archipelago in southeastern",Alaska,"The Lindenberg Peninsula is a peninsula on the eastern coast of Kupreanof Island, Alexander Archipelago in southeastern ʻŌhi"
6,between Kupreanof Island to the west and the,Alaska,between Kupreanof Island to the west and the ʻŌhi
7,Frederick Sound (also called Prince Frederick Sound or Prince Frederick's Sound) is a passage of water in the Alexander Archipelago in southeastern,Alaska,Frederick Sound (also called Prince Frederick Sound or Prince Frederick's Sound) is a passage of water in the Alexander Archipelago in southeastern ʻŌhi
8,Kupreanof Island is an island in the Alexander Archipelago in southeastern,Alaska,Kupreanof Island is an island in the Alexander Archipelago in southeastern ʻŌhi
9,Kupreanof Island is an island in the Alexander Archipelago in southeastern,Alaska,Kupreanof Island is an island in the Alexander Archipelago in southeastern ʻŌhi


In [20]:
from tqdm import tqdm
import spacy

nlp = spacy.load("en_core_web_sm")
lm_completions = []

for i, row in tqdm(lama_completion_df.iterrows(), total=lama_completion_df.shape[0]):
    try:
        prompt = row['prompt']
        completion = row['LM completion']

        lm_completions.append(completion.split(prompt)[1].strip())
    except Exception as e:
        lm_completions.append('')

100%|██████████| 100000/100000 [00:05<00:00, 19269.05it/s]


In [24]:
# detect entities generated by gpt

detected_ents = []
for doc in nlp.pipe(lm_completions, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
    detected_ents.append([(ent.text, ent.label_) for ent in doc.ents])
    

In [31]:
# classify detected generated entities based on their factualities
# if gpt fails to generate the ground-truth missing object and instead predicts 
# some other entities, mark these examples as hallucinations
# if gpt correctly generates a missing object, mark the example as factual

detected_ent_types = []
n_factual = 0
n_hallucinate = 0

for i, row in tqdm(lama_completion_df.iterrows(), total=lama_completion_df.shape[0]):
    if detected_ents[i]:
        if lm_completions[i] != '' and not row['true object'] in lm_completions[i]:
            detected_ent_types.append('hallucinated')
            n_hallucinate += 1
        elif lm_completions[i] != '' and row['true object'] in lm_completions[i]:
            detected_ent_types.append('factual')
            n_factual += 1
        else:
            detected_ent_types.append('NA')
    else:
        detected_ent_types.append('NA')
        
        

100%|██████████| 100000/100000 [00:04<00:00, 23604.60it/s]


In [32]:
n_factual

13701

In [33]:
n_hallucinate

39896

In [34]:
len(detected_ent_types)

100000

In [35]:
lama_completion_df['detected entities in completion'] = detected_ents
lama_completion_df['detected entities types'] = detected_ent_types

In [39]:
# collect all mentions of detected entities into a pandas df

detected_ent_df = {
    'lama dataset idx': [],
    'entity name': [],
    'entity factuality': []
}

for i in tqdm(range(len(detected_ents))):
    if detected_ent_types[i] != 'NA':
        for ent, _ in detected_ents[i]:
            detected_ent_df['lama dataset idx'].append(i)
            detected_ent_df['entity name'].append(ent)
            if lama_completion_df['true object'][i] == ent and detected_ent_types[i] == 'factual':
                detected_ent_df['entity factuality'].append('factual')
            else:
                detected_ent_df['entity factuality'].append('hallucination')

100%|██████████| 100000/100000 [00:00<00:00, 208819.16it/s]


In [40]:
detected_ent_df = pd.DataFrame(detected_ent_df)
detected_ent_df

,lama dataset idx,entity name,entity factuality
0,2,two,hallucination
1,3,two,hallucination
2,4,two,hallucination
3,11,California,hallucination
4,11,United States,hallucination
...,...,...,...
60339,99990,Ireland,hallucination
60340,99991,House of Commons,hallucination
60341,99993,1,hallucination
60342,99995,ʻU.S,hallucination


In [43]:
ent_factuality_type_count_df = detected_ent_df.groupby(['entity name']).agg({
    'entity factuality': lambda x: len(set(x))
}).reset_index()

In [46]:
# entities with both factual and hallucinated generations (call them target entities)

ent_factuality_type_count_df.loc[
    ent_factuality_type_count_df['entity factuality'] > 1
].reset_index()

,index,entity name,entity factuality
0,575,Afghanistan,2
1,576,Africa,2
2,643,Alabama,2
3,648,Alaska,2
4,658,Alberta,2
5,663,Algeria,2
6,699,Amsterdam,2
7,769,Argentina,2
8,775,Arkansas,2
9,779,Armenian,2


In [53]:


target_ents = set(ent_factuality_type_count_df.loc[
    ent_factuality_type_count_df['entity factuality'] > 1
]['entity name'])

# how many examples contain factual target entities?
n_target_completions_factual = len(detected_ent_df.loc[
    (detected_ent_df['entity name'].isin(target_ents)) &\
    (detected_ent_df['entity factuality'] == 'factual')
]['lama dataset idx'])

# how many examples contain hallucinated target entities?
n_target_completions_hallucinate = len(detected_ent_df.loc[
    (detected_ent_df['entity name'].isin(target_ents)) &\
    (detected_ent_df['entity factuality'] == 'hallucination')
]['lama dataset idx'])

In [54]:
n_target_completions_factual

12761

In [55]:
n_target_completions_hallucinate

15246

In [56]:
len(target_ents)

236